# If you think this notebook is worth reading and has gained some knowledge from this,please consider upvoting my kernel.Your appreciation means a lot to me

# Import required library

In [1]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import cv2
from tqdm import tqdm
import matplotlib.pyplot as plt

In [2]:
# Seed value
# Apparently you may use different seed values at each stage
seed_value= 48

# 1. Set the `PYTHONHASHSEED` environment variable at a fixed value
import os
os.environ['PYTHONHASHSEED']=str(seed_value)

# 2. Set the `python` built-in pseudo-random generator at a fixed value
import random
random.seed(seed_value)

# 3. Set the `numpy` pseudo-random generator at a fixed value
import numpy as np
np.random.seed(seed_value)

# 4. Set the `tensorflow` pseudo-random generator at a fixed value
import tensorflow as tf
# tf.random.set_seed(seed_value)
# for later versions: 
tf.compat.v1.set_random_seed(seed_value)

# 5. Configure a new global `tensorflow` session
# from keras import backend as K
# session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
# sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
# K.set_session(sess)

train and test data set

In [3]:
train=pd.read_csv('/kaggle/input/classification-of-images/dataset/train.csv')
test=pd.read_csv('/kaggle/input/classification-of-images/dataset/test.csv')
train.head()

,Image,Class
0,image7042.jpg,Food
1,image3327.jpg,misc
2,image10335.jpg,Attire
3,image8019.jpg,Food
4,image2128.jpg,Attire


convert categorical to numerical for training model

In [4]:

Class_map={'Food':0,'Attire':1,'Decorationandsignage':2,'misc':3}
inverse_map={0:'Food',1:'Attire',2:'Decorationandsignage',3:'misc'}
train['Class']=train['Class'].map(Class_map)

In [5]:
train['Class']

0       0
1       3
2       1
3       0
4       1
       ..
5978    0
5979    1
5980    0
5981    0
5982    1
Name: Class, Length: 5983, dtype: int64

In [7]:
train_img=[]
train_label=[]
j=0
path='/kaggle/input/classification-of-images/dataset/Train Images'
for i in tqdm(train['Image']):
    final_path=os.path.join(path,i)
    img=cv2.imread(final_path)
    img=cv2.resize(img,(128,128))
    img=img.astype('float32')
    img = img - [103.939, 116.779, 123.68]
    train_img.append(img)
    train_label.append(train['Class'][j])
    j=j+1

100%|██████████| 5983/5983 [00:13<00:00, 444.77it/s]


# Image agumentation

In [8]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

new_datagen = ImageDataGenerator()

datagen.fit(train_img)

In [9]:
test_img=[]
path='/kaggle/input/classification-of-images/dataset/Test Images'
for i in tqdm(test['Image']):
    final_path=os.path.join(path,i)
    img=cv2.imread(final_path)
    img=cv2.resize(img,(128,128))
    img=img.astype('float32')
    img = img - [103.939, 116.779, 123.68]
    test_img.append(img)

100%|██████████| 3219/3219 [00:07<00:00, 439.37it/s]


In [10]:
train_img=np.array(train_img)
test_img=np.array(test_img)
train_label=np.array(train_label)
print(train_img.shape)
print(test_img.shape)
print(train_label.shape)

(5728, 128, 128, 3)
(3219, 128, 128, 3)
(5728,)


In [11]:
from sklearn.utils.class_weight import compute_class_weight
class_weights = compute_class_weight('balanced', np.unique(train_label), train_label)
d_class_weights = dict(enumerate(class_weights))

In [22]:
from tensorflow.keras.applications.vgg16 import VGG16

from tensorflow.keras.layers import Flatten,Dense,Dropout,BatchNormalization
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.utils import to_categorical
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import ReduceLROnPlateau


# Vgg16 model

In [23]:
base_model=VGG16(include_top=False, weights='imagenet',input_shape=(128,128,3), pooling='avg')
model=Sequential()
model.add(base_model)
model.add(Dense(1024,activation='relu'))
model.add(Dense(4,activation='softmax'))


from keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop

base_model.trainable = False

reduce_learning_rate = ReduceLROnPlateau(monitor='loss',
                                         factor=0.1,
                                         patience=2,
                                         cooldown=2,
                                         min_lr=0.00001,
                                         verbose=1)

callbacks = [reduce_learning_rate]
    


model.compile( optimizer='adam'),loss='categorical_crossentropy',metrics=['accuracy'])

In [24]:
model.fit_generator(datagen.flow(train_img, to_categorical(train_label,4), batch_size=16),
                    epochs=30,callbacks=callbacks, class_weight=d_class_weights)

Train for 358 steps
Epoch 1/30
358/358 [==============================] - 26s 74ms/step - loss: 1.6408 - accuracy: 0.6163
Epoch 2/30
358/358 [==============================] - 22s 63ms/step - loss: 0.8799 - accuracy: 0.6789
Epoch 3/30
358/358 [==============================] - 22s 61ms/step - loss: 0.8799 - accuracy: 0.6805
Epoch 4/30
358/358 [==============================] - 23s 63ms/step - loss: 0.8331 - accuracy: 0.7057
Epoch 5/30
358/358 [==============================] - 22s 62ms/step - loss: 0.8097 - accuracy: 0.7200
Epoch 6/30
358/358 [==============================] - 22s 61ms/step - loss: 0.7492 - accuracy: 0.7394
Epoch 7/30
358/358 [==============================] - 23s 63ms/step - loss: 0.7589 - accuracy: 0.7453
Epoch 8/30
357/358 [============================>.] - ETA: 0s - loss: 0.7510 - accuracy: 0.7507
Epoch 00008: ReduceLROnPlateau reducing learning rate to 0.0009999999776482583.
358/358 [==============================] - 22s 62ms/step - loss: 0.7529 - accuracy: 0.7498

In [ ]:
model.fit_generator(datagen.flow(train_img, to_categorical(train_label,4), batch_size=16),
                    epochs=10,callbacks=callbacks, class_weight=d_class_weights)

# predict for test dataset

In [25]:
labels = model.predict(test_img)
print(labels[:4])
label = [np.argmax(i) for i in labels]
class_label = [inverse_map[x] for x in label]
print(class_label[:3])
submission = pd.DataFrame({ 'Image': test.Image, 'Class': class_label })
submission.head(10)
submission.to_csv('submission.csv', index=False)

[[9.6501023e-01 2.0476555e-02 1.1134567e-02 3.3786672e-03]
 [5.1619369e-01 6.1776731e-03 3.3043497e-04 4.7729814e-01]
 [9.3413365e-01 6.4882013e-05 2.3955388e-07 6.5801166e-02]
 [9.9999237e-01 1.3487572e-10 3.8153205e-13 7.6715914e-06]]
['Food', 'Food', 'Food']
